In [ ]:
import anndata
import scvelo as scv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc

: 

In [3]:
results_file = '/storage/holab/linxy/vivian/scanpy/ncc_seurate.h5ad'  # the file that will store the analysis results

In [4]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.8.0 pandas==1.4.2 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.9.10 pynndescent==0.5.6


In [5]:
# get meta-data information
path = "/storage/holab/linxy/vivian/veloAE/"
cellID_obs = pd.read_csv(path + "ncc_cellID.csv")
print(cellID_obs.head())
print(len(cellID_obs))
umap_cord = pd.read_csv(path + "ncc_cell_embeddings.csv")
print(umap_cord)
cell_clusters = pd.read_csv(path + "ncc_clusters.csv")

                     x
0  s1_AAACCCATCAGCAATC
1  s1_AAACGAACATTGTCGA
2  s1_AAAGGATCATGGCGCT
3  s1_AAAGGTAAGCGAATGC
4  s1_AAAGGTACAACTCGAT
10601
                Unnamed: 0    UMAP_1    UMAP_2
0      s1_AAACCCATCAGCAATC -0.249159  1.993144
1      s1_AAACGAACATTGTCGA -4.781256 -1.208898
2      s1_AAAGGATCATGGCGCT  5.002699  0.477262
3      s1_AAAGGTAAGCGAATGC  1.732118  2.226997
4      s1_AAAGGTACAACTCGAT -2.830450 -3.581742
...                    ...       ...       ...
10596  s4_TTTGGTTTCTCCTACG  0.079327  4.613376
10597  s4_TTTGTTGAGCATGATA  5.325161  3.589683
10598  s4_TTTGTTGCACGGCGTT -3.065098 -1.001408
10599  s4_TTTGTTGGTTTCCAAG  5.786833  2.374338
10600  s4_TTTGTTGTCATTGTTC -1.951135 -2.078952

[10601 rows x 3 columns]


In [7]:
sampleL = ["ncc1", "ncc2", "ncc3", "ncc4"]
adata = sc.read_10x_mtx('/storage/holab/linxy/vivian/' + sampleL[0] + '_cellranger/outs/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)                             
ldata =  scv.read("/storage/holab/linxy/vivian/" + sampleL[0] + "_cellranger/velocyto/" + sampleL[0] + "_cellranger.loom", cache=True)
adata.var_names_make_unique()
adata = scv.utils.merge(adata, ldata)
adata.obs_names = sampleL[0] + '_' + adata.obs_names
print(adata)
print(adata.obs_names)

... reading from cache file cache/storage-holab-linxy-vivian-ncc1_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc1_cellranger-velocyto-ncc1_cellranger.h5ad
AnnData object with n_obs × n_vars = 2506 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size'
    var: 'gene_ids', 'feature_types', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['ncc1_AAACCCATCAGCAATC', 'ncc1_AAACGAACACGGTGCT',
       'ncc1_AAACGAACATTGTCGA', 'ncc1_AAACGCTCAAGAGGTC',
       'ncc1_AAACGCTGTGCCCGTA', 'ncc1_AAAGAACCAGAGGTAC',
       'ncc1_AAAGAACTCTTGCAAG', 'ncc1_AAAGGATCACCATAAC',
       'ncc1_AAAGGATCATGGCGCT', 'ncc1_AAAGGGCTCTTGGCTC',
       ...
       'ncc1_TTTGACTTCAAGTGTC', 'ncc1_TTTGATCAGAGCGACT',
       'ncc1_TTTGGAGCATACAGAA', 'ncc1_TTTGGAGTCAAGTCGT',
       'ncc1_TTTGGTTGTTTGCCGG', 'ncc1_TTTGGTTTCTCCTACG',
       'n

In [6]:
for i in range(1,len(sampleL)):
    tempadata = sc.read_10x_mtx('/storage/holab/linxy/vivian/' + sampleL[i] + '_cellranger/outs/filtered_feature_bc_matrix/', var_names='gene_symbols', cache=True)                             
    templdata =  scv.read("/storage/holab/linxy/vivian/" + sampleL[i] + "_cellranger/velocyto/" + sampleL[i] + "_cellranger.loom", cache=True)
    tempadata.var_names_make_unique()
    tempadata = scv.utils.merge(tempadata, templdata)
    tempadata.obs_names = sampleL[i] + '_' + tempadata.obs_names
    adata = adata.concatenate(tempadata, join='outer', index_unique=None)

print(ldata)

... reading from cache file cache/storage-holab-linxy-vivian-ncc2_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc2_cellranger-velocyto-ncc2_cellranger.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


... reading from cache file cache/storage-holab-linxy-vivian-ncc3_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc3_cellranger-velocyto-ncc3_cellranger.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


... reading from cache file cache/storage-holab-linxy-vivian-ncc4_cellranger-outs-filtered_feature_bc_matrix-matrix.h5ad
... reading from cache file cache/storage-holab-linxy-vivian-ncc4_cellranger-velocyto-ncc4_cellranger.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 2506 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


## barcodes

In [11]:
## change obs_names
adata.obs_names = adata.obs_names.str.replace('ncc', 's')
print(adata.obs_names)
print(np.unique(adata.obs_names).size == len(adata.obs_names))

Index(['s1_AAACCCATCAGCAATC', 's1_AAACGAACACGGTGCT', 's1_AAACGAACATTGTCGA',
       's1_AAACGCTCAAGAGGTC', 's1_AAACGCTGTGCCCGTA', 's1_AAAGAACCAGAGGTAC',
       's1_AAAGAACTCTTGCAAG', 's1_AAAGGATCACCATAAC', 's1_AAAGGATCATGGCGCT',
       's1_AAAGGGCTCTTGGCTC',
       ...
       's1_TTTGACTTCAAGTGTC', 's1_TTTGATCAGAGCGACT', 's1_TTTGGAGCATACAGAA',
       's1_TTTGGAGTCAAGTCGT', 's1_TTTGGTTGTTTGCCGG', 's1_TTTGGTTTCTCCTACG',
       's1_TTTGTTGAGCATGATA', 's1_TTTGTTGAGGTTGTTC', 's1_TTTGTTGCACACGTGC',
       's1_TTTGTTGGTTTCCAAG'],
      dtype='object', length=2506)
True


In [9]:
## change cellID name in metadata
cellID_obs['x'] = cellID_obs['x'].str.replace('-1', '')
print(np.unique(cellID_obs['x']).size == len(cellID_obs['x']))  ## check whether have duplicate barcodes
print(len(cellID_obs))
print(cellID_obs)

True
10601
                         x
0      s1_AAACCCATCAGCAATC
1      s1_AAACGAACATTGTCGA
2      s1_AAAGGATCATGGCGCT
3      s1_AAAGGTAAGCGAATGC
4      s1_AAAGGTACAACTCGAT
...                    ...
10596  s4_TTTGGTTTCTCCTACG
10597  s4_TTTGTTGAGCATGATA
10598  s4_TTTGTTGCACGGCGTT
10599  s4_TTTGTTGGTTTCCAAG
10600  s4_TTTGTTGTCATTGTTC

[10601 rows x 1 columns]


In [12]:
filtered_barcode = np.intersect1d(cellID_obs['x'],adata.obs_names)
len(filtered_barcode)

1330

In [13]:
filtered_adata = adata[filtered_barcode].copy()
print(filtered_adata)

AnnData object with n_obs × n_vars = 1330 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size'
    var: 'gene_ids', 'feature_types', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


Add UMAP and annotation

In [14]:
adata_index = pd.DataFrame(filtered_adata.obs.index)
adata_index.columns = ['CellID']
print(adata_index)

                   CellID
0     s1_AAACCCATCAGCAATC
1     s1_AAACGAACATTGTCGA
2     s1_AAAGGATCATGGCGCT
3     s1_AAAGGTATCACCTGGG
4     s1_AAAGTCCAGAGATCGC
...                   ...
1325  s1_TTTGATCAGAGCGACT
1326  s1_TTTGGTTGTTTGCCGG
1327  s1_TTTGGTTTCTCCTACG
1328  s1_TTTGTTGAGCATGATA
1329  s1_TTTGTTGGTTTCCAAG

[1330 rows x 1 columns]


In [15]:
umap_cord = umap_cord.rename(columns = {'Unnamed: 0':'CellID'})
#umap_cord = umap_cord.rename(columns = {'Cell ID':'CellID'})
umap_cord['CellID'] = umap_cord['CellID'].str.replace('-1', '')
umap_ordered = adata_index.merge(umap_cord, on = "CellID")
print(umap_cord)

                    CellID    UMAP_1    UMAP_2
0      s1_AAACCCATCAGCAATC -0.249159  1.993144
1      s1_AAACGAACATTGTCGA -4.781256 -1.208898
2      s1_AAAGGATCATGGCGCT  5.002699  0.477262
3      s1_AAAGGTAAGCGAATGC  1.732118  2.226997
4      s1_AAAGGTACAACTCGAT -2.830450 -3.581742
...                    ...       ...       ...
10596  s4_TTTGGTTTCTCCTACG  0.079327  4.613376
10597  s4_TTTGTTGAGCATGATA  5.325161  3.589683
10598  s4_TTTGTTGCACGGCGTT -3.065098 -1.001408
10599  s4_TTTGTTGGTTTCCAAG  5.786833  2.374338
10600  s4_TTTGTTGTCATTGTTC -1.951135 -2.078952

[10601 rows x 3 columns]


In [16]:
umap_ordered = umap_ordered.iloc[:,1:]
filtered_adata.obsm['X_umap'] = umap_ordered.values

In [17]:
cell_clusters = cell_clusters.rename(columns = {'Unnamed: 0':'CellID'})
cell_clusters['CellID'] = cell_clusters['CellID'].str.replace('-1', '')
cell_clusters = adata_index.merge(cell_clusters, on = "CellID")
print(cell_clusters)

                   CellID  x
0     s1_AAACCCATCAGCAATC  5
1     s1_AAACGAACATTGTCGA  0
2     s1_AAAGGATCATGGCGCT  2
3     s1_AAAGGTATCACCTGGG  0
4     s1_AAAGTCCAGAGATCGC  0
...                   ... ..
1325  s1_TTTGATCAGAGCGACT  2
1326  s1_TTTGGTTGTTTGCCGG  4
1327  s1_TTTGGTTTCTCCTACG  5
1328  s1_TTTGTTGAGCATGATA  2
1329  s1_TTTGTTGGTTTCCAAG  2

[1330 rows x 2 columns]


In [18]:
cell_clusters = cell_clusters.iloc[:,1:]
filtered_adata.obs['seurat_clusters'] = cell_clusters.values
print(filtered_adata)

AnnData object with n_obs × n_vars = 1330 × 36601
    obs: 'Clusters', '_X', '_Y', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'seurat_clusters'
    var: 'gene_ids', 'feature_types', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


In [19]:
filtered_adata.write(results_file)